In [1]:
import torch
import numpy as np
import time
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 64
batch_size = 128
max_iters = 3000
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_layer = 8
n_head = 8
dropout = 0.2

cuda


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
chars = ""
with open('/content/drive/MyDrive/wizzard_of_oz.txt','r', encoding='utf-8') as f:
  text = f.read()
  chars = sorted(list(set(text)))

vocab_size = len(chars)

In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [4]:
data = torch.tensor(encode(text), dtype=torch.long)

In [5]:
block_size = 8
batch_size = 4

n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i:i+block_size] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

x, y = get_batch('train')
print("inputs:")
print(x)
print("targets:")
print(y)

inputs:
tensor([[75, 72, 77, 65, 82,  1, 60, 72],
        [77, 65, 62,  1, 51, 66, 83, 58],
        [69, 61,  1, 71, 72, 77,  1, 75],
        [78, 71, 77, 82, 66, 71, 64,  1]], device='cuda:0')
targets:
tensor([[75, 72, 77, 65, 82,  1, 60, 72],
        [77, 65, 62,  1, 51, 66, 83, 58],
        [69, 61,  1, 71, 72, 77,  1, 75],
        [78, 71, 77, 82, 66, 71, 64,  1]], device='cuda:0')


In [6]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train','val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = get_batch(split)
      logits, loss = model(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [7]:
class Head(nn.Module):
  """one head of self-attention"""

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))# save over haead computation
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    # input of size (batch, time-step, channels)
    # output of size (batch, time-step, head size)
    B, T, C = x.shape
    k = self.key(x) # (B,T,hs)
    q = self.query(x) # (B,T,hs)
    # compute attention scores ('affinities')
    wei = q @ k.transpose(-2,-1) * k.shape[-1] **-0.5 # (B, T, hs) @ (B, hs, T) - > (B,B,T)
    wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim=-1) # (B, T, T)
    wei = self.dropout(wei)
    v = self.value(x)
    out = wei @ v
    return out

class MultiHeadAttention(nn.Module):
  """Multiple heads of self-attention in parallel"""

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.head = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(head_size * num_heads, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.head], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2 ,h2, h3, h3, h3, h3])
    out = self.dropout(self.proj(out))
    return out


class FeedForward(nn.Module):
  """ a simple linear layer followed by a non-linearity"""
  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd),
        nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

class Block(nn.Module):
  """ Transformer block: communication followed by computation """
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    y = self.sa(x) #self attention
    x = self.ln1(x+y) # add a norm
    y = self.ffwd(x) # feed forward net
    x = self.ln2(x + y) # add a norm
    return x

class GPTLanguageModel(nn.Module):

  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
      torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
      if module.bias is not None:
        torch.nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
        torch.nn.init.normal_(module.weight, mean=0.0, std=0.2)

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd) #final layer norm
    self.lm_head = nn.Linear(n_embd, vocab_size)
    self.apply(self._init_weights)


  def forward(self, index, targets=None):
    B, T = index.shape
    #idx and targets are both (B,T) tensor of integers
    tok_emb = self.token_embedding_table(index)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb #(B,T,C)
    x = self.blocks(x) #(B,T,C) encoders
    x = self.ln_f(x) #(B,T,C) final layer neuron
    logits = self.lm_head(x) #(B,T, vocab_size) end softmax decision head
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, index, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self.forward(index)
      logits = logits[:,-1,:]
      probs = F.softmax(logits, dim=-1)
      index_next = torch.multinomial(probs, num_samples=1)
      index = torch.cat((index, index_next), dim=1)

    return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)


In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
  if iter % eval_iters == 0:
    losses = estimate_loss()
    print(f"step {iter}, losses {losses}")

  xb, yb = get_batch('train')
  logits, loss = model.forward(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

step 0, losses {'train': tensor(4.6188), 'val': tensor(4.6143)}
step 100, losses {'train': tensor(0.0639), 'val': tensor(0.1021)}
step 200, losses {'train': tensor(0.0178), 'val': tensor(0.0506)}
step 300, losses {'train': tensor(0.0067), 'val': tensor(0.0391)}
step 400, losses {'train': tensor(0.0031), 'val': tensor(0.0153)}
step 500, losses {'train': tensor(0.0055), 'val': tensor(0.0166)}
step 600, losses {'train': tensor(0.0015), 'val': tensor(0.0203)}
step 700, losses {'train': tensor(0.0016), 'val': tensor(0.0232)}
step 800, losses {'train': tensor(0.0028), 'val': tensor(0.0145)}
step 900, losses {'train': tensor(0.0016), 'val': tensor(0.0254)}
step 1000, losses {'train': tensor(0.0008), 'val': tensor(0.0151)}
step 1100, losses {'train': tensor(0.0006), 'val': tensor(0.0203)}
step 1200, losses {'train': tensor(0.0014), 'val': tensor(0.0184)}
step 1300, losses {'train': tensor(0.0004), 'val': tensor(0.0065)}
step 1400, losses {'train': tensor(0.0006), 'val': tensor(0.0166)}
step 15

Transformers
General transformer architecture. A lot of computation going on. Feed forward networks.